In [ ]:
#get the data
!wget https://www.gutenberg.org/files/1661/1661-0.txt -o book.txt

In [ ]:
import tensorflow as tf
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
text = open('1661-0.txt','r',encoding='utf-8').read()
text = text.lower()


In [ ]:
sentences = text.split("\n")

In [ ]:
tokenizer = Tokenizer(oov_token = '<UNK>')

In [ ]:
tokenizer.fit_on_texts(sentences)

In [ ]:
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
sequences = tokenizer.texts_to_sequences(sentences)
print(tokenizer.word_index)

{'<UNK>': 1, 'the': 2, 'and': 3, 'to': 4, 'of': 5, 'a': 6, 'i': 7, '”': 8, 'in': 9, 'that': 10, 'it': 11, 'he': 12, 'was': 13, 'you': 14, 'his': 15, 'is': 16, 'my': 17, 'have': 18, 'with': 19, 'as': 20, 'had': 21, 'at': 22, 'which': 23, 'for': 24, 'be': 25, 'not': 26, 'me': 27, 'but': 28, 'from': 29, 'we': 30, 'this': 31, 'said': 32, 'upon': 33, 'there': 34, 'holmes': 35, 'him': 36, 'so': 37, 'her': 38, 'she': 39, 'all': 40, '’': 41, 'been': 42, 'your': 43, 'on': 44, 'very': 45, 'by': 46, 'one': 47, 'are': 48, '“i': 49, 'were': 50, 'an': 51, 'no': 52, 'would': 53, 'out': 54, 'what': 55, 'then': 56, 'up': 57, 'when': 58, 'man': 59, 'could': 60, 'has': 61, 'do': 62, 'into': 63, 'or': 64, 'little': 65, 'will': 66, 'who': 67, 'mr': 68, 'if': 69, 'some': 70, 'down': 71, 'see': 72, 'now': 73, 'our': 74, 'should': 75, 'may': 76, 'am': 77, 'us': 78, 'over': 79, 'they': 80, 'can': 81, 'more': 82, 'think': 83, 'about': 84, 'must': 85, 'shall': 86, 'know': 87, 'room': 88, 'before': 89, 'any': 90,

In [ ]:
input_sequences = []
for seq in sequences:
  for i in range(1,len(seq)):
    n_gram_seq = seq[:i+1]
    input_sequences.append(n_gram_seq)

In [ ]:
max_seq_len = max([len(seq) for seq in input_sequences])

20


In [ ]:
padded_sequences = pad_sequences(input_sequences,maxlen = max_seq_len )

In [ ]:
import numpy as np
padded_sequneces = np.array(padded_sequences)

In [ ]:
x = padded_sequences[:,:-1]
labels = padded_sequences[:,-1]
y = tf.keras.utils.to_categorical(labels, num_classes = vocab_size)

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from tensorflow.keras.optimizers import Adam

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length = max_seq_len-1))
model.add(Bidirectional(LSTM(256)))
model.add(Dense(vocab_size,activation = 'softmax'))
adam  = Adam(learning_rate = 0.01)
model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['acc'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 19, 100)           893300    
_________________________________________________________________
bidirectional (Bidirectional (None, 512)               731136    
_________________________________________________________________
dense (Dense)                (None, 8933)              4582629   
Total params: 6,207,065
Trainable params: 6,207,065
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor = 'acc', min_delta = 0.001)

In [ ]:
model.fit(x,y,verbose = 1, batch_size = 512, epochs=50, callbacks=[es])

Epoch 1/50
199/199 [==============================] - 15s 77ms/step - loss: 1.3617 - acc: 0.6447
Epoch 2/50
199/199 [==============================] - 15s 76ms/step - loss: 1.3183 - acc: 0.6561
Epoch 3/50
199/199 [==============================] - 15s 76ms/step - loss: 1.2941 - acc: 0.6588
Epoch 4/50
199/199 [==============================] - 15s 77ms/step - loss: 1.2864 - acc: 0.6600
Epoch 5/50
199/199 [==============================] - 15s 76ms/step - loss: 1.2766 - acc: 0.6597


In [ ]:
seed_text = "I could not help laughing at the sense with which he explained his process of deduction"
next_words = 100

for _ in range(next_words):
  sequence = tokenizer.texts_to_sequences([seed_text])
  padded = pad_sequences(sequence,maxlen = max_seq_len-1)

  predicted = model.predict_classes(padded ,verbose = 0)
  output_word= ''
  for word, index in tokenizer.word_index.items():
    if(index == predicted):
      output_word = word
      break
  seed_text += ' '+ output_word
print(seed_text)

[[   0    0    0    7   60   26  259  662   22    2 1292   19   23   12
  1213   15 1949    5 1695]]
[[   0    0    7   60   26  259  662   22    2 1292   19   23   12 1213
    15 1949    5 1695   24]]
[[   0    7   60   26  259  662   22    2 1292   19   23   12 1213   15
  1949    5 1695   24   34]]
[[   7   60   26  259  662   22    2 1292   19   23   12 1213   15 1949
     5 1695   24   34   48]]
[[  60   26  259  662   22    2 1292   19   23   12 1213   15 1949    5
  1695   24   34   48  384]]


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


[[  26  259  662   22    2 1292   19   23   12 1213   15 1949    5 1695
    24   34   48  384  568]]
[[ 259  662   22    2 1292   19   23   12 1213   15 1949    5 1695   24
    34   48  384  568    9]]
[[ 662   22    2 1292   19   23   12 1213   15 1949    5 1695   24   34
    48  384  568    9    2]]
[[  22    2 1292   19   23   12 1213   15 1949    5 1695   24   34   48
   384  568    9    2  128]]
[[   2 1292   19   23   12 1213   15 1949    5 1695   24   34   48  384
   568    9    2  128   11]]
[[1292   19   23   12 1213   15 1949    5 1695   24   34   48  384  568
     9    2  128   11   16]]
[[  19   23   12 1213   15 1949    5 1695   24   34   48  384  568    9
     2  128   11   16    6]]
[[  23   12 1213   15 1949    5 1695   24   34   48  384  568    9    2
   128   11   16    6   45]]
[[  12 1213   15 1949    5 1695   24   34   48  384  568    9    2  128
    11   16    6   45 2024]]
[[1213   15 1949    5 1695   24   34   48  384  568    9    2  128   11
    16    6   45 20

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


I could not help laughing at the sense with which he explained his process of deduction for there are several points in the morning it is a very brave for it is a good of mr neville st clair of the daughter as to the nature of the noble bachelor may have had the great friend i may do though the very great claret i have lost that i was saving a intimate friend and ran with a very shiny hat and the anxiety in the morning however i dropped my gun and held the hand and then it was the appearance of the morning however and finally the red and director of this sort in
